In [1]:
# First, we'll import pandas, a data processing and CSV file I/O library
import pandas as pd
from scipy import special
import matplotlib as mpl
mpl.use('Agg')
#from matplotlib.mlab import PCA as mlabPCA
import numpy as np
import time
#from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from scipy.special import log1p

from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import VarianceThreshold
# We'll also import seaborn, a Python graphing library
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
import hypertools as hyp
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
sns.set(style="white", color_codes=True)
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.learning_curve import learning_curve
from sklearn.cross_validation import cross_val_predict
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.lda import LDA

# The data csv file is given as features as rows and data as columns. 
# sets IlmnID as index to avoid default indexing. This is needed to avoid another column names while transposing.
# Reads all columns now(data)
# Reads beta file. Reads manifest. drops rs collumns, sex probes and NA features.
def readMedeComReconstructed():
    beta_rec_old = pd.read_csv("data/reconstructedbeta.csv", index_col = 0) #, usecols= [*range(0, 261)], index_col = 0')  
    #beta = beta_rec_old.T # Transposes to have data along rows
    beta_recon = beta_rec_old.transpose().reset_index().rename(columns={'index':'Composite_Element_REF'})
    beta_recon.columns.name = None 
    #Drops columns with 'rs' features from beta file
    beta_recon = beta_recon.drop(beta_recon.filter(like='rs').columns, 1)  
    print(beta_recon.shape)
    #Drops columns with Sex probes(X and Y). Reads from manifestfile
    #From manifest sees the column CHR and finds X or Y. If yes, return a list that changed to DF.
    # Now make it series data of list.
    manifest = pd.read_csv("data/HumanMethylation450_15017482_v1-2.csv")
    sex_probes_df = pd.DataFrame(manifest.loc[(manifest['CHR'] == 'X') | (manifest['CHR'] == 'Y')], columns =['IlmnID'])
    sex_probes = sex_probes_df['IlmnID'].values.tolist()
    beta_recon = beta_recon.drop(beta_recon.filter(sex_probes).columns, 1)
    print(beta_recon.shape)
    # Drops NaN data if any are NaN (The data has almost all data value NaN for some features)
    beta_recon=beta_recon.dropna(axis='rows',how='any')
    print(beta_recon.shape)
    return beta_recon
def readBetaFile():
    sarc_old = pd.read_csv("data/SARC-combined.txt", usecols= [*range(0, 261)], index_col = 0, sep='\t')  
    #sarc = sarc_old.T # Transposes to have data along rows
    sarc = sarc_old.transpose().reset_index().rename(columns={'index':'Composite_Element_REF'})
    sarc.columns.name = None 
    #print(sarc.shape)
    #Drops columns with 'rs' features from beta file
    sarc = sarc.drop(sarc.filter(like='rs').columns, 1)  
    #print(sarc.shape)
    #Drops columns with Sex probes(X and Y). Reads from manifestfile
    #From manifest sees the column CHR and finds X or Y. If yes, return a list that changed to DF.
    # Now make it series data of list.
    manifest = pd.read_csv("data/HumanMethylation450_15017482_v1-2.csv")
    sex_probes_df = pd.DataFrame(manifest.loc[(manifest['CHR'] == 'X') | (manifest['CHR'] == 'Y')], columns =['IlmnID'])
    sex_probes = sex_probes_df['IlmnID'].values.tolist()
    sarc = sarc.drop(sarc.filter(sex_probes).columns, 1)
    #print(sarc.shape)
    # Drops NaN data if any are NaN (The data has almost all data value NaN for some features)
    sarc=sarc.dropna(axis='columns',how='any')
    #print(sarc.shape)
    return sarc
#Reads clinical file and compares it gainst the beta file for Ids to find labels for betas
#merges data with corresponding labels
#Modifies wrong labels. Drops Despoid tumours.
#Drops the outlier
def mergeClinicalFile(keepall):
    clinic = pd.read_csv("data/TCGA_SARC_Clinical-2.csv", sep='\s*,\s*')
    #clinic = pd.read_csv("dummy.csv", sep='\s*,\s*')
    clinic.rename(columns={'Composite Element REF':'Composite_Element_REF'}, inplace=True)
    #prints index column calues
    #print(clinic.index.values)
    #print feature names
    #print(list(clinic))
    #print index values and corresponding values against the feature - "Residual Tumor"
    #print(clinic.loc[: , "residual_tumor"])
    # setting header to the columns of interests
    header = ["Composite_Element_REF","residual_tumor"]
    # saving the index column and residual_columns data to sample.csv
    #clinic.to_csv("sample.csv", columns= header)
    
    # BEAUTIFUL - changes column values instantly, takes fist 10 values, replace . with _ and upper case
    merge_clinic = pd.DataFrame(clinic, columns = ['Composite_Element_REF', 'residual_tumor'] )
    #merge_clinic.to_csv("clinic_ID_label.csv")
    #merge_clinic['Composite_Element_REF'] = merge_clinic['Composite_Element_REF'].apply(lambda x: str(x)[0:12].replace('.', '_').upper())
    #print(merge_clinic['Composite_Element_REF'])
    #Just to see the data(not needed for project)
    print_original = pd.DataFrame(sarcoma, columns = ['Composite_Element_REF'] )
    #print_original.to_csv("original_beta.csv")
    if(keepall==0):
        merge_sarcoma = pd.DataFrame(sarcoma, columns = ['Composite_Element_REF'] )
    else:
        print("HHHHHHH")
        merge_sarcoma = pd.DataFrame(sarcoma)
    merge_sarcoma['Composite_Element_REF'] = merge_sarcoma['Composite_Element_REF'].apply(lambda x: str(x)[0:12].replace('-', '.').lower())
    #Just to see the data(not needed for project)
    analysis_sarc_data = pd.DataFrame(merge_sarcoma, columns = ['Composite_Element_REF'])
    #analysis_sarc_data.to_csv("betafile_data_Ids.csv")
    #df3 = merge_clinic.merge(merge_sarcoma,on='Composite_Element_REF',
                  #how='outer').dropna(subset=['residual_tumor'])
    #compares the cloumns for identity and merges sarcoma data set with labels    
    result = pd.merge(merge_sarcoma, merge_clinic, on='Composite_Element_REF', how='inner')
    # Just for printing to test(not used for project)
    analysis_data = pd.DataFrame(result, columns = ['Composite_Element_REF', 'residual_tumor'] )
    #analysis_data.to_csv("id_label_not_combined.csv")
    #frames = [merge_clinic, merge_sarcoma]
    #result = pd.concat(frames)
    #result.to_csv("sample.csv")
    
    result['residual_tumor'] = result.residual_tumor.str.replace(r'(^.*undifferentiated.*$)', 'Undifferentiated Sarcoma')
    result['residual_tumor'] = result.residual_tumor.str.replace(r'(^.*synovial.*$)', 'Synovial Sarcoma')
    
    #Modify Wrong lables in clinical file
    result.loc[result['Composite_Element_REF'] == 'tcga.hb.a3yv', 'residual_tumor'] = 'dedifferentiated liposarcoma'
    result.loc[result['Composite_Element_REF'] == 'tcga.pc.a5dn', 'residual_tumor'] = 'dedifferentiated liposarcoma'
    result.loc[result['Composite_Element_REF'] == 'tcga.dx.a6yz', 'residual_tumor'] = 'dedifferentiated liposarcoma'
    result.loc[result['Composite_Element_REF'] == 'tcga.3b.a9hs', 'residual_tumor'] = 'dedifferentiated liposarcoma'
    result.loc[result['Composite_Element_REF'] == 'tcga.dx.ab2s', 'residual_tumor'] = 'dedifferentiated liposarcoma'
    # Drop Desmoid Tumors
    result = result.drop(result.index[result.residual_tumor == 'desmoid tumor']) #tcga.z4.a8jb, tcga.qq.a8vd
    result.index = range(len(result)) # Rearrange index
    #Delete the outlier    
    result = result.drop(result.index[result.Composite_Element_REF == 'tcga.k1.a6rt']) #tcga.k1.a6r
    
    result.index = range(len(result)) # Rearrange index
    # Just for printing to test(not used for project)
    analysis_data = pd.DataFrame(result, columns = ['Composite_Element_REF', 'residual_tumor'] )
      
    #analysis_data.to_csv("id_label_combined.csv")
    #Drop ID
    result = result.drop('Composite_Element_REF', axis=1)
    
    return result

# Print values
def printValues():
    # prints the index column values. Those are Ids related to the data. Not used for calculation                                           
    print(sarc.index.values)
    print(sarc.index)
    # prints the values of data. Index column and column headers are ignored by default. its same as sarc. but an array.
    print(sarc.values)
    #print feature names except index column
    print(list(sarc))
# Low variance feature reduction
def lowVarianceFeatureReduction():
    selector = VarianceThreshold(0.09)
    columns = sarcoma_X.columns
    sarc_new = selector.fit_transform(sarcoma_X)
    column_names = [columns[x] for x in selector.get_support(indices=True) if x]
    newdata = pd.DataFrame(selector.fit_transform(sarcoma_X), columns=column_names)
    return newdata
#PCA with fixed components
def pcaWithFixedComp(noOfcomp):
    pca = PCA(n_components=noOfcomp)
    pca_result = pca.fit_transform(sarcoma_X)    
    trainx, testx, trainy, testy = train_test_split(pca_result, sarcoma_Y, test_size=0.3, random_state=0, stratify=sarcoma_Y)
    seed = 7
    #fit model no training data
    model = XGBClassifier()
    model.fit(trainx, trainy)
 
    y_test_pred = model.predict(testx)    
    cm_tst = confusion_matrix(testy, y_test_pred)
    print(cm_tst)
    #Plot CM for test data
    
    plt.clf()
    ax = sns.heatmap(cm_tst,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    plt.savefig('plots/pca_xgboost_cf.png', bbox_inches='tight')
    #Print accuracy report
    report = classification_report(testy, y_test_pred)
    df = classifaction_report_csv(report)
    df.to_csv('plots/pca_xgboost_report.csv', index = True)
    print(classification_report(testy, y_test_pred))
    print(accuracy_score(testy, y_test_pred))

# PCA transformation with TSNE - One practice is use PCA to reduce to 100 then apply tsne to 2 and visualize
def pcaWithTSNE():
    n_sne = 50
    pca = PCA()
    pca.fit(sarcoma_X)
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    d = np.argmax(cumsum>=0.95) + 1
    pca = PCA(n_components = 0.95)
    pca_result = pca.fit_transform(sarcoma_X)
    print(sum(pca.explained_variance_ratio_))
    #TSNE calculation
    
    for i in [2,3, 4, 5, 7, 8, 10,20,30,40,50,60,70,80,90,100,200]:
        X_tsne = TSNE(learning_rate=100, n_components=3, verbose=1, 
                  perplexity=i, n_iter=10000).fit_transform(pca_result)   
        #Visualize tsne and pca
        hyp.plot(X_tsne,'o', group=sarcoma_Y, legend=label_names, title = 'TSNE PLOT AFTER PCA PER %s'%i, palette="deep")
    #TSNE after Truncated SVD used in sparse data. First redude to less dimensions then tsne.
    from sklearn.decomposition import TruncatedSVD
    X_reduced = TruncatedSVD(n_components=50, random_state=0).fit_transform(sarcoma_X)
    for j in [2,3,4,5,7,10,20,30,40,50,60,70,80,90,100,200]:
        X_embedded = TSNE(n_components=3, perplexity=j, verbose=2, n_iter=10000).fit_transform(X_reduced)
        hyp.plot(X_embedded,'o', group=sarcoma_Y, legend=label_names, title = 'SVD TSNE PLOT PER %s'%j, palette="deep")
    ####################
    T_SNE = TSNE(learning_rate=100, n_components=3, verbose=1, 
                  perplexity=40, n_iter=50000).fit_transform(sarcoma_X)
    hyp.plot(T_SNE,'o', group=sarcoma_Y, legend=label_names, title = 'TSNE OVER FULL DIMENSIONS', palette="deep")
    ##########################
    from sklearn.manifold import MDS
    MDS_model = MDS(n_components=3, n_jobs=-1).fit_transform(sarcoma_X)
    hyp.plot(T_SNE,'o', group=sarcoma_Y, legend=label_names, title = 'MDS OVER FULL DIMENSIONS', palette="deep")
    return pca_result
#Correlation matrix
def pca_split():
    n_sne = 50
    pca = PCA()
    pca.fit(sarcoma_X)
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    d = np.argmax(cumsum>=0.95) + 1
    pca = PCA(n_components = 0.95)
    pca_result = pca.fit_transform(sarcoma_X)
    return pca_result
def findCorr(pca_result):
    #NumPy Array, convert to Dataframe
    corr = pd.DataFrame(pca_result).corr(method='pearson')   
    sns.heatmap(corr, 
        xticklabels=corr.columns.values,
        yticklabels=corr.columns.values)
    plt.show()
#Kernal PCA
def logisticRegressionSoftmax(pca_result, result):
    print(pca_result)
    
    trainx, testx, trainy, testy = train_test_split(pca_result, sarcoma_Y, test_size=0.3, random_state=0, stratify=sarcoma_Y)
    softmax_reg = linear_model.LogisticRegression(multi_class = "multinomial", solver = "lbfgs", C = 10)
    softmax_reg.fit(trainx, trainy)
    y_test_pred = softmax_reg.predict(testx)
    accuracy = accuracy_score(testy, y_test_pred)
        
    cm_tst = confusion_matrix(testy, y_test_pred)
    
    ax = sns.heatmap(cm_tst,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    
    plt.savefig('plots/log_reg_cm.png', bbox_inches='tight')
    #Print accuracy report
    report = classification_report(testy, y_test_pred)
    df = classifaction_report_csv(report)
    df.to_csv('plots/logregre_report.csv', index = True)
    print(accuracy_score(testy, y_test_pred))
    print(classification_report(testy, y_test_pred))
    
def kerPCA():
    rbf_pca = KernelPCA(n_components = 2, kernel = "rbf", gamma = 0.03)
    X_red =  rbf_pca.fit_transform(sarcoma_M_X)
    plot_figures(X_red, 'plots/kerPCA.png', "Kernel PCA")
#printValues()
def summaryStats(pca_result):
    #pca_result.head() # return the first 5 rows
    #pca_result.describe() # summary statistics, excluding NaN values
    #pca_result.info(verbose=True, null_counts=True) # concise summary of the table
    #pca_result.shape # shape of dataset
    print(pca_result.skew()) # skewness for numeric columns
    #pca_result.kurt() # unbiased kurtosis for numeric columns
    #pca_result.get_dtype_counts() # counts of dtypes
    
#Encode lables with numerica values and remove the header
def encodeLabels(datavalue):
    le.fit(datavalue['residual_tumor'])
    datavalue['residual_tumor'] = le.transform(datavalue['residual_tumor'])
    #labels = datavalue['residual_tumor'] # returns only labels exclusing header
    return datavalue
def labelBinarize(result):
    result = label_binarize(y, classes=[0, 1, 2, 3, 4, 5,6, 7])
    

def plotCorrTsnePCsnLabels(nComponent):
    sns.set(style="darkgrid")
    sns.swarmplot(sarcoma_Y,xtsne[:,nComponent],order=[0, 1, 2, 3, 4, 5, 6])
    plt.title('Correlation between PCs and target')
    plt.xlabel('Target')
    plt.ylabel('PC Value')
    plt.show()
#Plots desnity plot - to see the skewness
def plotDensityTsne():
    pd.DataFrame(xtsne).plot(kind='density', subplots=True, layout=(3,3), sharex=False)
    plt.show()
def printXandY():
    print(sarcoma_X)
    print(sarcoma_Y)
#Plots density of mean, mad etc along data(mean and mad is of each category of cancer seperate)
def plotDensityAlongDataOrFeat(sarcoma_with_label_codes):
    group_by_lables = sarcoma_with_label_codes.groupby(sarcoma_with_label_codes['residual_tumor']).mad()
    count_of_labels = sarcoma_with_label_codes[['residual_tumor']].groupby('residual_tumor').size()
    group_by_mean = sarcoma_with_label_codes.groupby(sarcoma_with_label_codes['residual_tumor']).mean()
    for col_id in range(1000):
        #sns.distplot(group_by_mean.iloc[:,col_id]) #Plots along individual features
        sns.distplot(group_by_mean.iloc[col_id]) # Plots along data
    plt.show()
    
def tsne_scatter():
    
    #Pca
    pca = PCA()
    pca.fit(sarcoma_X)
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    d = np.argmax(cumsum>=0.95) + 1
    pca = PCA(n_components = 0.95)
    pc1 = pca.fit(sarcoma_X)
    pca_result = pca.fit_transform(sarcoma_X)
    print(pc1.n_components_)
    print(pc1.explained_variance_ratio_)
    X_tsne = TSNE(learning_rate=100, n_components=2, verbose=1, 
                  perplexity=60, n_iter=10000).fit_transform(sarcoma_X) 
    plot_figures(X_tsne, 'plots/tsne.png', 'TSNE(without PCA)')
    
def classify_LDA(no_of_components, type_of_cancer_label):
    lda = LinearDiscriminantAnalysis(n_components=no_of_components)
    lda_transformed = lda.fit_transform(sarcoma_X, sarcoma_Y)
    #print(lda_transformed)
    #lda_csv = pd.DataFrame(lda_transformed)
    #lda_csv.to_csv("lda_csv.csv")
    #y_csv = pd.DataFrame(sarcoma_Y)
    #y_csv.to_csv("y_csv.csv")
    #print(type(sarcoma_Y))
    #print(y_nump)
    #hyp.plot(lda_transformed,'o', legend=label_names, title = 'LDA OVER FULL DIMENSIONS', palette="deep")
    #plt.scatter(lda_transformed[:,0], lda_transformed[:,1], c=y,cmap=plt.cm.coolwarm, label = label_names)
    #plt.legend(loc='best', shadow=False, scatterpoints=None)
    plot_figures(lda_transformed, 'plots/lda.png', "LDA")
def classify_LDA_Test():
    lda = LinearDiscriminantAnalysis(n_components=2) # Creating LDA object
    lda = lda.fit(X_train, y_train) # Create the transformation matrix
    lda_train = lda.transform(X_train) # Transforms X train data
    plt.clf()
    colors = ['magenta', 'cyan', 'grey', 'red', 'green', 'orange']
    fig = plt.figure()
    ax = plt.subplot(111)
    y_nump =y_train.values
    title = "LDA TRAIN DATA"
    fileName = "lda_split_train.png"
    for color, i, target_name in zip(colors, [0, 1, 2, 3, 4, 5], label_names):
        ax.scatter(lda_train[y_nump == i, 0], lda_train[y_nump == i, 1], alpha=.8, color=color,label=target_name)
    
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    lgd = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5)) 
    ax.grid('on')
    plt.title('%s plot of Sarcoma Dataset'%title)
    plt.savefig(fileName, bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
    test_labels = lda.predict(X_test)
    plt.clf()
    colors = ['magenta', 'cyan', 'grey', 'red', 'green', 'orange']
    fig = plt.figure()
    ax = plt.subplot(111)
    y_nump =test_labels.values
    title = "LDA TEST DATA"
    fileName = "lda_split_test.png"
    for color, i, target_name in zip(colors, [0, 1, 2, 3, 4, 5], label_names):
        ax.scatter(lda_test[y_nump == i, 0], lda_test[y_nump == i, 1], alpha=.8, color=color,label=target_name)
    
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    lgd = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5)) 
    ax.grid('on')
    plt.title('%s plot of Sarcoma Dataset'%title)
    plt.savefig(fileName, bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show() 
       
    cm_tst = confusion_matrix(y_test, test_labels)
    print(cm_tst)
    #Plot CM for test data
    ax = sns.heatmap(cm_tst,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    
    plt.savefig('plots/lda_test_CM.png', bbox_inches='tight')
    plt.show()
    
def classifaction_report_csv(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        row['cancer type'] = row_data[1]
        row['precision'] = float(row_data[2])
        row['recall'] = float(row_data[3])
        row['f1_score'] = float(row_data[4])
        row['support'] = float(row_data[5])
        report_data.append(row)
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe
def plot_figures(data_to_plot, fileName, title):
    #Plot params
    plt.clf()
    colors = ['magenta', 'cyan', 'grey', 'red', 'green', 'orange']
    fig = plt.figure()
    ax = plt.subplot(111)
    y_nump =sarcoma_Y.values
    for color, i, target_name in zip(colors, [0, 1, 2, 3, 4, 5], label_names):
        ax.scatter(data_to_plot[y_nump == i, 0], data_to_plot[y_nump == i, 1], alpha=.8, color=color,label=target_name)
    
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    lgd = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5)) 
    ax.grid('on')
    plt.title('%s plot of Sarcoma Dataset'%title)
    plt.savefig(fileName, bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
    
def classify_linearSVC():
    #Find hyperparameter for Linear SVC based on accuracy
    
    svc = LinearSVC(penalty="l1", dual=False) # trains 6 models of shape(_samples, n_classes)
    cv = StratifiedKFold(n_splits=5)
    C_s = np.logspace(-3, 1, 30)  #10^-3 to 10^ 1 with 10 steps. so its 0.001 to 1 
    scores = list()
    scores_std = list()
    for C in C_s:
        svc.C = C
        #cross validation accuracy. 
        this_scores = cross_val_score(svc, X_train, y_train, cv = 4, n_jobs=1)
        scores.append(np.mean(this_scores))
        scores_std.append(np.std(this_scores))
    # Do the plotting
    plt.figure(1, figsize=(4, 3))
    #plt.clf()
    plt.semilogx(C_s, scores)
    plt.semilogx(C_s, np.array(scores) + np.array(scores_std), 'b--')
    plt.semilogx(C_s, np.array(scores) - np.array(scores_std), 'b--')
    locs, labels = plt.yticks()
    plt.yticks(locs, list(map(lambda x: "%g" % x, locs)))
    plt.ylabel('CV score')
    plt.xlabel('Parameter C')
    plt.ylim(0, 1.1)
    plt.savefig('plots/linearsvc_CV_error.png')
    plt.show()
    # Another way to meaure accuracy of training with cross validation, but here we took C = best. 
    # we can do the same above to find C
    
    plt.clf()
    best_C = C_s[np.argmax(scores)]
    #Just check the prediction efficiency against best C for 5 fold cross validated data
    svc.C = best_C
    y_train_pred = cross_val_predict(svc,X_train,y_train, cv=4)   
    cm = confusion_matrix(y_train, y_train_pred)
    # Plot confusion metrix against train prediction cv   
    ax = sns.heatmap(cm,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    
    plt.savefig('plots/linearSVC_train.png', bbox_inches='tight')
    plt.show()
    plt.clf()
    np.fill_diagonal(cm, 0)
    ax = sns.heatmap(cm,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    
    plt.savefig('plots/linearSVC_train_filldiag.png', bbox_inches='tight')
    plt.show()
    plt.clf()
    #Fit model to the train data with best C value
    svc.fit(X_train, y_train)
    y_test_pred = svc.predict(X_test)
    cm_tst = confusion_matrix(y_test, y_test_pred)
    print(cm_tst)
    #Plot CM for test data
    ax = sns.heatmap(cm_tst,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    
    plt.savefig('plots/linearSVC_test.png', bbox_inches='tight')
    plt.show()
    plt.clf()
    np.fill_diagonal(cm_tst, 0)
    ax = sns.heatmap(cm_tst,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    
    plt.savefig('plots/linearSVC_test_filldiag.png', bbox_inches='tight')
    plt.show()
    #Print accuracy report
    report = classification_report(y_test, y_test_pred)
    df = classifaction_report_csv(report)
    df.to_csv('plots/linearSVC_report.csv', index = True)
    print(classification_report(y_test, y_test_pred))
    print(accuracy_score(y_test, y_test_pred))
    
    #cm_perf = confusion_matrix(y_train, y_train_perfect_predictions)
def classify_NonlinearSVC():
    #lsvc = LinearSVC(C=1000, penalty="l1", dual=False).fit(sarcoma_X, sarcoma_Y)
    #model = SelectFromModel(lsvc, prefit=True)
    #X_new = model.transform(sarcoma_X)
    #print(X_new.shape)
    svc = SVC(C=1000, kernel="rbf")
    n_samples = X_train.shape[0]
    cv = ShuffleSplit(n_samples, n_iter=3, test_size=0.3, random_state=0)
    score = cross_val_score(svc, X_train, y_train, cv=cv)
    print(score)
def classify_linearSVC_CV_autom():
    #--Split into Train and test set 80/20
    #X_train, X_test, y_train, y_test = train_test_split(sarcoma_X, sarcoma_Y, test_size=0.2, random_state=0, stratify=sarcoma_Y)
    #--Choose estimator
    estimator = SVC(kernel="rbf")
    #--Choose Cross validation estimator
    cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0)
    #-- Tune Hyperparameters
    gammas = np.logspace(-6, -1, 10)
    classifier = GridSearchCV(estimator=estimator, cv=cv, param_grid=dict(gamma=gammas))
    classifier.fit(X_train, y_train)
    #-- Debug algorithm with learning curve
    title = 'Learning Curves (SVM, linear kernel, $\gamma=%.6f$)' %classifier.best_estimator_.gamma
    estimator = SVC(kernel='rbf', gamma=classifier.best_estimator_.gamma)
    plot_learning_curve(estimator, title, X_train, y_train, cv=cv)
    plt.show()
    #pred = lsvc.predict(X_test)
    
def classifyXGBoost():
    seed = 7
    #fit model no training data
    model = XGBClassifier()
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    cm = confusion_matrix(y_train, y_train_pred)
    # Plot confusion metrix against train prediction cv   
    ax = sns.heatmap(cm,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    plt.savefig('plots/xgboost_train_cf.png', bbox_inches='tight')
    
    plt.clf()
    y_test_pred = model.predict(X_test)
    predictions = [round(value) for value in y_test_pred]
    #evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))    
    cm_tst = confusion_matrix(y_test, y_test_pred)
    #Plot CM for test data
    
    ax = sns.heatmap(cm_tst,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    
    plt.savefig('plots/xgboost_test_cf.png', bbox_inches='tight')
    #Print accuracy report
    report = classification_report(y_test, y_test_pred)
    df = classifaction_report_csv(report)
    df.to_csv('plots/xgboost_report.csv', index = True)
    print(accuracy_score(y_test, y_test_pred))
    print(classification_report(y_test, y_test_pred))
# Does cross validation, takes parameter tuning, use SVC or lR based on loss etc
def classify_linearSGD():
    tuned_parameters = {'alpha': [10 ** a for a in range(-4, -2)], 'l1_ratio' : [1 * a for a in range(0, 1)] }
    # Important for tunedparameters in grdsearch
    #tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
    #                 'C': [1, 10, 100, 1000]},
    #                {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    #SGDclassifier with loss = 'hinge' means its SVC  if loss = 'log' its logistic classfier
    clf = GridSearchCV(SGDClassifier(loss='hinge', penalty='elasticnet',
                    n_iter=400, 
                    shuffle=True, verbose=False, n_jobs=10, average=False, class_weight='balanced'),
                    tuned_parameters, cv=10, scoring='f1_macro')
    clf.fit(X_train, y_train)
    print(clf.best_params_) 
def classify_RandomForestClassifier():
    #rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=1000, oob_score = True)
    #param_grid = { 
        #'n_estimators': [5000],
        #'max_features': ['auto', 'sqrt', 'log2']
    #}
    #CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
    #CV_rfc.fit(lowVSar_X, sarcoma_Y)
    #print(CV_rfc.best_params_)
    #print(CV_rfc.best_score_)
    
    '''rfc1 = RandomForestClassifier(oob_score = True)
    rfc1.fit(X_train, y_train)
    param_grid = { "n_estimators"      : [4000, 5000, 6000, 7000, 8000],
           "criterion"         : ["gini", "entropy"],
           "max_features"      : [10, 20, 30, 40, 50],
           "max_depth"         : [10, 20, 30, 40, 50],
           "min_samples_split" : [10, 20, 30, 40] ,
           "bootstrap": [True, False]}
    grid_search = GridSearchCV(rfc1, param_grid, n_jobs=-1, cv=2)
    grid_search.fit(X_train, y_train)
    print(grid_search.best_params_)'''
    
    
    #print(list(zip(sarcoma_X, clf.feature_importances_)))
    #importances = forest.feature_importances_
    
    #X_train, X_test, y_train, y_test = train_test_split(features, sarcoma_Y, test_size=0.3, random_state=0, stratify=sarcoma_Y)
    rfc = RandomForestClassifier(n_jobs=-1,max_features= 'auto' ,n_estimators=8000, oob_score = True)
    rfc.fit(X_train, y_train)
    
    y_train_pred = rfc.predict(X_train)
    cm = confusion_matrix(y_train, y_train_pred)
    # Plot confusion metrix against train prediction cv   
    ax = sns.heatmap(cm,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    plt.savefig('plots/rf_train_cf.png', bbox_inches='tight')
       
    y_test_pred = rfc.predict(X_test)    
    cm_tst = confusion_matrix(y_test, y_test_pred)
    #print(cm_tst)
    #Plot CM for test data
    plt.clf()
    ax = sns.heatmap(cm_tst,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    plt.savefig('plots/rf_test_cf.png', bbox_inches='tight')
    #Print accuracy report
    report = classification_report(y_test, y_test_pred)
    df = classifaction_report_csv(report)
    df.to_csv('plots/rf_report.csv', index = True)
    print(classification_report(y_test, y_test_pred))
    print(accuracy_score(y_test, y_test_pred))
def test():
    forest = RandomForestClassifier(n_estimators=1000)
    forest.fit(X_train, y_train)
    #print(list(zip(sarcoma_X, clf.feature_importances_)))
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_train.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
    plt.xticks(range(X_train.shape[1]), indices)
    plt.xlim([-1, X_train.shape[1]])
    plt.show()
def Univariate_ChiSquare_XGBoost():
    test = SelectKBest(score_func=chi2, k=130000)
    
    fit = test.fit(sarcoma_X, sarcoma_Y)
    #numpy.set_printoptions(precision=3)
    #print(fit.scores_)
    features = fit.transform(sarcoma_X)
    print(features.shape)
    
    seed = 7
    
    trainx, testx, trainy, testy = train_test_split(features, sarcoma_Y, test_size=0.3, random_state=0, stratify=sarcoma_Y)
    
    #fit model no training data
    model = XGBClassifier()
    model.fit(trainx, trainy)
    
    y_test_pred = model.predict(testx)    
    cm_tst = confusion_matrix(testy, y_test_pred)
    print(cm_tst)
    #Plot CM for test data
    
    plt.clf()
    ax = sns.heatmap(cm_tst,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    plt.savefig('chisquare_test_cf.png', bbox_inches='tight')
    #Print accuracy report
    report = classification_report(testy, y_test_pred)
    df = classifaction_report_csv(report)
    df.to_csv('chsq_report.csv', index = True)
    print(classification_report(testy, y_test_pred))
    print(accuracy_score(testy, y_test_pred))
    
def getLabelCounts(labels):
    df = pd.DataFrame(labels)
    print(df.residual_tumor.value_counts())
    
def Univariate_ChiSquare():
    chi_square = SelectKBest(score_func=chi2, k=20000)    
    fit = chi_square.fit(sarcoma_X, sarcoma_Y)
    features = fit.transform(sarcoma_X)
    return features
def classify_RF_PCA(noOfcomp):
   
    pca = PCA(n_components=noOfcomp)
    pca_result = pca.fit_transform(sarcoma_X)
    
    trainx, testx, trainy, testy = train_test_split(pca_result, sarcoma_Y, test_size=0.3, random_state=0, stratify=sarcoma_Y)
    
    seed = 7
    rfc = RandomForestClassifier(n_jobs=-1,max_features= 'auto' ,n_estimators=8000, oob_score = True)
    rfc.fit(trainx, trainy)
    
    
    y_test_pred = rfc.predict(testx)    
    cm_tst = confusion_matrix(testy, y_test_pred)
    #print(cm_tst)
    #Plot CM for test data
    plt.clf()
    ax = sns.heatmap(cm_tst,annot=True, xticklabels=label_names, yticklabels=label_names)
    ax.set_ylabel('Actual classes')
    ax.set_xlabel('Predicted classes')
    plt.savefig('plots/pca_rf_test_cf.png', bbox_inches='tight')
    #Print accuracy report
    report = classification_report(testy, y_test_pred)
    df = classifaction_report_csv(report)
    df.to_csv('plots/pca_rf_report.csv', index = True)
    print(classification_report(testy, y_test_pred))
    print(accuracy_score(testy, y_test_pred))
#Mislabels data in clinical file need to be changed
    
    
# Function calls starts . Reads beta file.
sarcoma = readBetaFile()
#Reads reconstructed MedCom file
#sarcoma = readMedeComReconstructed()
#Result df contains only label data if parameter passed is 0. else it returns a df with all merged with labels
# features names, index, label values. But not the Composite_Element_REF. Its dropped.
sarcoma_with_label_names = mergeClinicalFile(1)
#Use only for MedeCom file
#sarcoma_with_label_names.to_csv("reconstr_beta_withlabel.csv")
#print(sarcoma_with_label_names.shape)
#print(getLabelCounts(sarcoma_with_label_names)) # Good one to show no of labels for each
#Returns encoded same as result one. with encoded labels. Would say this is the final data 
#with features and labels headings as well.
le = LabelEncoder()
sarcoma_with_label_codes = encodeLabels(sarcoma_with_label_names)
#print(sarcoma_with_label_codes.shape)
label_names = list(le.inverse_transform([0, 1, 2, 3, 4, 5]))
print(label_names)

sarcoma_Y= sarcoma_with_label_codes['residual_tumor']

sarcoma_X = sarcoma_with_label_codes.drop('residual_tumor', axis=1)

sarcoma_X_scale = scale(sarcoma_X)
sarcoma_M_X = special.logit(sarcoma_X)
print(sarcoma_X)
#Feature select with variance
#pca_result = pca_split() # Use with LogisticRegressionSoftmax()
features = Univariate_ChiSquare() # Use with LogisticRegressionSoftmax()
#features = lowVarianceFeatureReduction() # Use with LogisticRegressionSoftmax()
X_train, X_test, y_train, y_test = train_test_split(sarcoma_X, sarcoma_Y, test_size=0.3, random_state=0, stratify=sarcoma_Y)
#Use only when we use feature selection algorithms
#X_train, X_test, y_train, y_test = train_test_split(features, sarcoma_Y, test_size=0.3, random_state=0, stratify=sarcoma_Y)
#print(X_train)
#lowVSar_X = lowVarianceFeatureReduction()
#print("Full Set of Labels below ******************")
#getLabelCounts(sarcoma_Y)
#print("Train Set of Labels below ******************")
#getLabelCounts(y_test) # Print label counts
#getLabelCounts(y_train) # Print label counts
#print("Test Set of Labels below ******************")
#getLabelCounts(y_test)

#X_train, X_test, y_train, y_test = train_test_split(lowVSar_X, sarcoma_Y, test_size=0.30, random_state=0, stratify=sarcoma_Y)

#printXandY()

#pcaWithFixedComp(10000)
#summaryStats(pd.DataFrame(pca_result))
############

#plotDensityTsne()
###############
#plotCorrTsnePCsnLabels(0)
#plotCorrTsnePCsnLabels(1)
#plotCorrTsnePCsnLabels(2)
#sns.violinplot(sarcoma_data.iloc[0], color = 'cyan')
#plt.show()
#findCorr(pca_result) 
#plotDensityAlongDataOrFeat(sarcoma_with_label_codes)
#classify_RF_PCA(10000)
#classify_LDA_Test()
####################### Classifiers #######################
#kerPCA()
#classify_LDA(2, 7) # 2 is no of components, 7 means plots all labels else plots specific type of cancer only
#tsne_scatter()
print("-------Linear SVC ------------")
#classify_linearSVC()# 71.25 with medecom
#print("-------RF ------------")
#classify_RandomForestClassifier() #77.5% with medecom and n_estimators 8000(>5000)
#print("-------XG Boost ------------")
#classifyXGBoost() #  72.5 with medecom

#classify_linearSGD()
#classify_NonlinearSVC()
#########################




#test()
#######################
logisticRegressionSoftmax(features, sarcoma_Y)
##### FEATURE SELECTION - RF Classifier



#svc = SVC(kernel="linear")
#rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
              #scoring='accuracy')
#rfecv.fit(X_train, y_train)

#print(rfecv.n_features_)

#Plot number of features VS. cross-validation scores
#plt.figure()
#plt.xlabel("Number of features selected")
#plt.ylabel("Cross validation score (nb of correct classifications)")
#plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
#plt.show()

HHHHHHH
['Synovial Sarcoma', 'Undifferentiated Sarcoma', 'dedifferentiated liposarcoma', 'leiomyosarcoma (lms)', 'malignant peripheral nerve sheath tumors (mpnst)', 'myxofibrosarcoma']
     cg00000165  cg00000236  cg00000289  cg00000292  cg00000321  cg00000363  \
0      0.822568    0.922048    0.750420    0.377120    0.069381    0.805901   
1      0.235924    0.906233    0.612593    0.922852    0.089916    0.251482   
2      0.814205    0.895598    0.617794    0.752132    0.746498    0.410345   
3      0.416769    0.893083    0.656475    0.668648    0.528341    0.399737   
4      0.136909    0.812579    0.624955    0.666994    0.330979    0.120983   
5      0.705386    0.887615    0.569937    0.796094    0.506325    0.273498   
6      0.503227    0.905776    0.616173    0.421955    0.171419    0.238287   
7      0.187319    0.872477    0.611403    0.766338    0.410168    0.368802   
8      0.243633    0.837798    0.565070    0.818292    0.234684    0.093653   
9      0.114265    0.8974